In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader 
from abc import ABC
from tqdm import tqdm
import torchvision
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random




config = {}
config["seed"] = 42
seed = config["seed"]
os.environ['PYTHONHASHSEED'] = str(seed)
# Torch RNG
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Python RNG
np.random.seed(seed)
random.seed(seed)




config["num_clients_per_cluster"] = 10
config["num_clusters"] = 2
config["num_clients"] = config["num_clients_per_cluster"]*config["num_clusters"]
config["dataset"] = "cifar10"
DATASET_LIB = {"mnist" : torchvision.datasets.MNIST, "emnist": torchvision.datasets.EMNIST, "cifar10": torchvision.datasets.CIFAR10}
config["dataset_dir"] = "./experiments/dataset"
config["results_dir"] = "./experiments/results"
config["results_dir"] = os.path.join(config["results_dir"], config["dataset"])

os.makedirs(config["results_dir"], exist_ok=True)

train_dataset = DATASET_LIB[config["dataset"]](root = config['dataset_dir'], download = True, train=True)
test_dataset = DATASET_LIB[config["dataset"]](root = config['dataset_dir'], download = True, train=False)




Files already downloaded and verified
Files already downloaded and verified


In [3]:
def split(dataset_size, num_clients, shuffle):
    split_idx = []
    all_idx = np.arange(dataset_size)
    if shuffle:
        all_idx = np.random.permutation(all_idx)
    split_idx = np.array_split(all_idx, num_clients)
    return split_idx

def dataset_split(train_data, test_data, num_clients, shuffle):
    train_size = train_data[0].shape[0]
    train_split_idx = split(train_size, num_clients, shuffle)
    train_chunks = [(train_data[0][train_split_idx[client]], train_data[1][train_split_idx[client]]) for client in range(num_clients)]
    test_size = test_data[0].shape[0]
    test_split_idx = split(test_size, num_clients, shuffle)
    test_chunks = [(test_data[0][test_split_idx[client]], test_data[1][test_split_idx[client]]) for client in range(num_clients)]
    return train_chunks, test_chunks

def make_client_datasets(config):
    train_chunks_total = []
    test_chunks_total = []
    train_dataset = DATASET_LIB[config["dataset"]](root = config['dataset_dir'], download = True, train=True)
    test_dataset = DATASET_LIB[config["dataset"]](root = config['dataset_dir'], download = True, train=False)

    train_data = (train_dataset.data, np.array(train_dataset.targets))
    test_data = (test_dataset.data, np.array(test_dataset.targets))
    for i in range(config["num_clusters"]):
        train_chunks, test_chunks = dataset_split(train_data, test_data, config["num_clients_per_cluster"], shuffle=True)
        train_chunks_total += train_chunks
        test_chunks_total += test_chunks
    return train_chunks_total, test_chunks_total


class ClientWriteDataset(Dataset):
    def __init__(self, data):
        super(ClientWriteDataset,self).__init__()
        self.data = data[0]
        self.labels = data[1]

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        idx_data = self.data[idx]
        idx_labels = self.labels[idx]
        return (idx_data, idx_labels)


In [ ]:
class ClientDataset(Dataset):
    def __init__(self, data,transforms = None):
        super(ClientDataset,self).__init__()
        self.data = data[0]
        self.labels = data[1]
        self.transforms = transforms

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        idx_data = self.data[idx]
        if self.transforms is not None:
            transformed_data =self.transforms(idx_data)
        else:
            transformed_data = idx_data
        idx_labels = self.labels[idx]
        return (transformed_data, idx_labels)


In [15]:
class Client():
    def __init__(self, train_data, test_data, client_id,  train_transforms, test_transforms, train_batch_size, test_batch_size, save_dir):
        self.trainset = ClientDataset(train_data)
        self.testset = ClientDataset(test_data)
        self.trainloader = DataLoader(self.trainset, batch_size = train_batch_size, shuffle=True, num_workers=8)
        self.testloader = DataLoader(self.testset, batch_size = test_batch_size, shuffle=False, num_workers=8)
        self.train_iterator = iter(self.trainloader)
        self.test_iterator = iter(self.testloader)
        self.client_id = client_id
        self.save_dir = os.path.join(save_dir, "init", "client_{}".format(client_id))
        self.save_dir

    def sample_batch(self, train=True):
        iterator = self.train_iterator if train else self.test_iterator
        try:
            (data, labels) = next(iterator)
        except StopIteration:
            if train:
                self.train_iterator = iter(self.trainloader)
                iterator = self.train_iterator
            else:
                self.test_iterator = iter(self.testloader)
                iterator = self.test_iterator
            (data, labels) = next(iterator)
        return (data, labels)


In [16]:
train_chunks, test_chunks = make_client_datasets(config)


Files already downloaded and verified
Files already downloaded and verified


In [17]:
import torchvision.transforms as transforms



config["train_batch"] = 512
config["test_batch"] = 512
client_loaders = []
train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_transforms = transforms.Compose([
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

for i in range(config["num_clusters"]):
    for j in range(config["num_clients_per_cluster"]):
        idx = i * config["num_clients_per_cluster"] + j
        x_train = torch.as_tensor(train_chunks[idx][0]).float()
        x_test = torch.as_tensor(test_chunks[idx][0]).float()
        x_train = x_train.permute(0,3,1,2)
        x_test = x_test.permute(0,3,1,2)

        if i >0:
            x_train = torch.rot90(x_train, i*2, [2, 3])
            x_test = torch.rot90(x_test, i*2, [2, 3])
        client_loaders.append(
            Client(
                (x_train, torch.as_tensor(train_chunks[idx][1])),
                (x_test, torch.as_tensor(test_chunks[idx][1])),
                idx,
                train_transforms=train_transforms,
                test_transforms=None,
                train_batch_size=config["train_batch"],
                test_batch_size=config["test_batch"],
                save_dir=config["results_dir"],
            )
        )


Traceback (most recent call last):
  File "/home/harshv834/anaconda3/envs/ffcv/lib/python3.9/multiprocessing/queues.py", line 251, in _feed
    send_bytes(obj)
  File "/home/harshv834/anaconda3/envs/ffcv/lib/python3.9/multiprocessing/connection.py", line 205, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/harshv834/anaconda3/envs/ffcv/lib/python3.9/multiprocessing/connection.py", line 416, in _send_bytes
    self._send(header + buf)
  File "/home/harshv834/anaconda3/envs/ffcv/lib/python3.9/multiprocessing/connection.py", line 373, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/harshv834/anaconda3/envs/ffcv/lib/python3.9/multiprocessing/queues.py", line 251, in _feed
    send_bytes(obj)
  File "/home/harshv834/anaconda3/envs/ffcv/lib/python3.9/multiprocessing/connection.py", line 205, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/harshv834/anaconda3/

In [19]:
from torchvision.models import resnet18
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.model = resnet18(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad=False
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, 10)
        
    def forward(self, input):
        x = self.model(input)
        return x
    
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        layer_list = [self.conv1, self.conv2, self.pool, self.fc1, self.fc2]
        for layer in layer_list:
            for param in layer.parameters():
                param.requires_grad = False
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x




512

In [ ]:
# def train_fedavg_model(train_chunks, test_chunks, config):
# model = SimpleLinear()
# model.to(torch.device("cuda:0"))
# train_data = []
# train_labels = [chunk[1] for chunk in train_chunks]
# test_labels = [chunk[1] for chunk in test_chunks]
# test_data = []
# for i in range(config["num_clusters"]):
#     for j in range(config['num_clients_per_cluster']):
#         idx = i*config["num_clients_per_cluster"] + j
#         train_data.append()
# train_set = ClientDataset(data)

In [21]:
def calc_acc(model, device, client_data, train):
    loader = client_data.trainloader if train else client_data.testloader
    model.eval()
    acc = 0
    with torch.no_grad():
        for (X,Y) in loader:
            X = X.to(device)
            pred = model(X).argmax(axis=1).detach().cpu()
            acc += (Y == pred).float().mean()
    acc = acc/len(loader)
    acc *= 100.0
    return acc

import time
class BaseTrainer(ABC):
    def __init__(self,config, save_dir):
        super(BaseTrainer, self).__init__()
        self.model = MODEL_LIST[config["model"]](**config["model_params"])
        self.save_dir = save_dir
        self.device = config["device"]
        self.loss_func = LOSSES[config["loss_func"]]
        self.config = config
        os.makedirs(self.save_dir, exist_ok=True)

    def train(self):
        raise NotImplementedError
    
    def test(self):
        raise NotImplementedError

    def load_model_weights(self):
        model_path  = os.path.join(self.save_dir, "model.pth")
        if os.path.exists(model_path):
            self.model.load_state_dict(torch.load(model_path))
        else:
            print("No model present at path : {}".format())

    def save_model_weights(self):
        model_path  = os.path.join(self.save_dir, "model.pth")
        torch.save(self.model.state_dict(), model_path)
    def save_metrics(self, train_loss, test_acc):
        torch.save({"train_loss": train_loss,  "test_acc" : test_acc}, os.path.join(self.save_dir,"metrics.pkl"))

class ClientTrainer(BaseTrainer):
    def __init__(self,  config, save_dir,client_id):
        super(ClientTrainer, self).__init__(config, save_dir)
        self.client_id = client_id
    
    def train(self, client_data):
        train_loss_list = []
        test_acc_list = []
        self.model.to(self.device)
        self.model.train()
        optimizer = OPTIMIZER_LIST[self.config["optimizer"]](self.model.parameters(), **self.config["optimizer_params"])
        for iteration in tqdm(range(self.config["iterations"])):
            t0 = time.time()
            self.model.zero_grad()
            (X,Y) = client_data.sample_batch(train=True)
            X = X.to(self.device)
            Y = Y.to(self.device)
            out = self.model(X)
            loss = self.loss_func(out, Y)
            loss.backward()
            optimizer.step()
            train_loss = loss.detach().cpu().numpy().item()
            train_loss_list.append(train_loss)
            test_acc = calc_acc(self.model, self.device, client_data, train=False)
            test_acc_list.append(test_acc)
            self.model.train()
            t1 = time.time()
            time_taken = t1 - t0
            if iteration % self.config["save_freq"] == 0 or iteration == self.config["iterations"] - 1:
                self.save_model_weights()
                self.save_metrics(train_loss_list, test_acc_list)
                print("Iteration : {} \n , Train Loss : {} \n, Test Acc : {} \n, Time : {}\n".format(iteration,  train_loss, test_acc, time_taken))
                
        self.model.eval()
        self.model.cpu()


    def test(self, client_data):
        self.load_model_weights()
        self.model.eval()
        self.model.to(self.device)
        acc =  calc_acc(self.model, client_data)
        self.model.cpu()
        return acc


  
MODEL_LIST = {"resnet" : ResNet, "cnn":Net}
OPTIMIZER_LIST = {"sgd": optim.SGD, "adam": optim.Adam}
LOSSES = {"cross_entropy": nn.CrossEntropyLoss()}
# config["save_dir"] = os.path.join("./results")
config["iterations"] = 20000
config["optimizer_params"] = {"lr":0.001}
config["save_freq"] = 10
config["model"] = "cnn"
config["optimizer"] = "adam"
config["loss_func"] = "cross_entropy"
#config["model_params"] = {"num_channels": 1 , "num_classes"  : 62}
config["model_params"] = {}
config["device"] = torch.device("cuda:0")
import pickle
client_trainers = [ClientTrainer(config,os.path.join(config["results_dir"], "init", "node_{}".format(i)), i) for i in range(config["num_clients"])]


for i in tqdm(range(config["num_clients"])):
    client_trainers[i].train(client_loaders[i])
    


  0%|          | 0/20 [00:00<?, ?it/s]

Iteration : 0 
 , Train Loss : 9.119747161865234 
, Test Acc : 11.489818572998047 
, Time : 0.4879488945007324



Iteration : 10 
 , Train Loss : 4.583766460418701 
, Test Acc : 13.457351684570312 
, Time : 0.5162496566772461



Iteration : 20 
 , Train Loss : 4.011845588684082 
, Test Acc : 9.8008451461792 
, Time : 0.5425963401794434



Iteration : 30 
 , Train Loss : 3.6533756256103516 
, Test Acc : 14.505955696105957 
, Time : 0.5324900150299072



Iteration : 40 
 , Train Loss : 3.2456367015838623 
, Test Acc : 13.593429565429688 
, Time : 0.5237395763397217



Iteration : 50 
 , Train Loss : 3.156414270401001 
, Test Acc : 13.012295722961426 
, Time : 0.5247807502746582



  0%|          | 0/20 [00:34<?, ?it/s]


KeyboardInterrupt: 

In [78]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(range(config["num_clients"]))
import itertools
def model_weights_diff(w_1, w_2):
    norm_sq = 0
    assert w_1.keys() == w_2.keys(), "Model weights have different keys"
    for key in w_1.keys():
        norm_sq  += (w_1[key] - w_2[key]).norm()**2
    return np.sqrt(norm_sq)
thresh = 0
wt = client_trainers[0].model.state_dict()
for key in wt.keys():
    thresh += wt[key].norm()**2
thresh = 37.68

all_pairs = list(itertools.combinations(range(config["num_clients"]),2))
for pair in all_pairs:
    w_1  = client_trainers[pair[0]].model.state_dict()
    w_2 = client_trainers[pair[1]].model.state_dict()
    norm_diff = model_weights_diff(w_1, w_2)
    if norm_diff < thresh:
        G.add_edge(pair[0], pair[1])
        
cliques = list(nx.algorithms.clique.enumerate_all_cliques(G))


RuntimeError: norm(): input dtype should be either floating point or complex dtypes. Got Long instead.

In [64]:
config["t"] = 7
t = config["t"]
clusters = [clique  for clique in cliques if len(clique) > t]
cluster_map = {i: clusters[i] for i in range(len(clusters))}
beta = 0.2

In [73]:
class ClusterTrainer(BaseTrainer):
    def __init__(self,  config, save_dir,cluster_id):
        super(ClusterTrainer, self).__init__(config, save_dir)
        self.cluster_id = cluster_id
    
    def train(self, client_data_list):
        num_clients = len(client_data_list)

        train_loss_list = []
        test_acc_list = []
        self.model.to(self.device)
        self.model.train()
        optimizer = OPTIMIZER_LIST[self.config["optimizer"]](self.model.parameters(), **self.config["optimizer_params"])
        for iteration in tqdm(range(self.config["iterations"])):

            trmean_buffer = {}
            for idx, param in self.model.named_parameters():
                trmean_buffer[idx] = []
            train_loss = 0
            for client in client_data_list:
                optimizer.zero_grad()
                (X,Y) = client.sample_batch()
                X = X.to(config["device"])
                Y = Y.to(config["device"])
                loss_func = nn.CrossEntropyLoss()
                out = self.model(X)
                loss = loss_func(out,Y)
                loss.backward()
                train_loss += loss.detach().cpu().numpy().item()
                with torch.no_grad():
                    for idx, param in self.model.named_parameters():
                        trmean_buffer[idx].append(param.grad.clone())

            optimizer.zero_grad()
            for idx, param in self.model.named_parameters():
                sorted, _  = torch.sort(torch.stack(trmean_buffer[idx], dim=0), dim=0)
                new_grad = sorted[int(beta*num_clients):int((1-beta)*num_clients),...].mean(dim=0)
                param.grad = new_grad
                trmean_buffer[idx] = []
            optimizer.step()
            
            train_loss_list.append(train_loss/num_clients)
            test_acc = 0
            for client_data in client_data_list:
                test_acc += calc_acc(self.model, self.device, client_data, train=False)
            test_acc = test_acc/num_clients
            test_acc_list.append(test_acc)
            self.model.train()
            if iteration % self.config["save_freq"] == 0 or iteration == self.config["iterations"] - 1:
                self.save_model_weights()
                self.save_metrics(train_loss_list, test_acc_list)
                print("Iteration : {} \n , Train Loss : {} \n, Test Acc : {} \n".format(iteration,  train_loss, test_acc))
                
        self.model.eval()
        self.model.cpu()


    def test(self, client_data_list):
        self.load_model_weights()
        self.model.eval()
        self.model.to(self.device)
        test_acc = 0
        for client_data in client_data_list:
            test_acc += calc_acc(self.model, self.device, client_data, train=False)
        self.model.cpu()
        return test_acc


In [78]:
%%time 
config["refine_steps"] = 3
for refine_step in tqdm(range(config["refine_steps"])):
    beta = 0.2
    cluster_trainers = []
    for cluster_id in tqdm(cluster_map.keys()):
        cluster_clients = [client_loaders[i] for i in cluster_map[cluster_id]]
        cluster_trainer = ClusterTrainer(config, os.path.join(config['results_dir'], "refine_{}".format(refine_step), "cluster_{}".format(cluster_id)), cluster_id)
        cluster_trainer.train(cluster_clients)
        cluster_trainers.append(cluster_trainer)
    with open(os.path.join(config["results_dir"],"refine_{}".format(refine_step), "cluster_maps.pkl"), 'wb') as handle:
        pickle.dump(cluster_map, handle, protocol=pickle.HIGHEST_PROTOCOL)
    cluster_map_recluster = {}
    for key in cluster_map.keys():
        cluster_map_recluster[key] = []

    for i in tqdm(range(config["num_clients"])):
        w_node = client_trainers[i].model.state_dict()
        norm_diff = np.infty
        new_cluster_id = 0
        for cluster_id in cluster_map.keys():
            w_cluster = cluster_trainers[cluster_id].model.state_dict()
            curr_norm_diff = model_weights_diff(w_node, w_cluster)
            if norm_diff > curr_norm_diff:
                new_cluster_id = cluster_id
                norm_diff = curr_norm_diff
        
        cluster_map_recluster[new_cluster_id].append(i)
    cluster_map = cluster_map_recluster


    G = nx.Graph()
    G.add_nodes_from(cluster_map.keys())

    all_pairs = list(itertools.combinations(cluster_map.keys(),2))
    for pair in tqdm(all_pairs):
        w_1  = cluster_trainers[pair[0]].model.state_dict()
        w_2 = cluster_trainers[pair[1]].model.state_dict()
        norm_diff = model_weights_diff(w_1, w_2)
        if norm_diff < thresh:
            G.add_edge(pair[0], pair[1])
            
    clusters = list(nx.algorithms.clique.enumerate_all_cliques(G))
    cluster_map_new = {}
    for i in range(len(clusters)):
        cluster_map_new[i] = []
        for j in clusters[i]:
            cluster_map_new[i] += cluster_map[j]
    cluster_map = cluster_map_new


  0%|          | 0/157 [00:00<?, ?it/s]

Iteration : 0 
 , Train Loss : 161.17435836791992 
, Test Acc : 21.751054763793945 



Iteration : 2 
 , Train Loss : 668.6630401611328 
, Test Acc : 27.509302139282227 



Iteration : 4 
 , Train Loss : 966.4324493408203 
, Test Acc : 31.116979598999023 



Iteration : 6 
 , Train Loss : 703.5293579101562 
, Test Acc : 39.541622161865234 



...

Iteration : 14 
 , Train Loss : 246.76281356811523 
, Test Acc : 47.5426139831543 



Iteration : 16 
 , Train Loss : 156.58898258209229 
, Test Acc : 60.839542388916016 



Iteration : 18 
 , Train Loss : 107.62056732177734 
, Test Acc : 63.467533111572266 



100%|██████████| 157/157 [22:45<00:00,  8.70s/it]

Iteration : 19 
 , Train Loss : 102.45139598846436 
, Test Acc : 62.21410369873047 

CPU times: user 22min 42s, sys: 16.1 s, total: 22min 58s
Wall time: 22min 45s


In [83]:
with open(os.path.join(config["results_dir"],"refine_{}".format(refine_step), "cluster_maps.pkl"), 'wb') as handle:
    pickle.dump(cluster_map, handle, protocol=pickle.HIGHEST_PROTOCOL)
cluster_map_recluster = {}
for key in cluster_map.keys():
    cluster_map_recluster[key] = []

for i in tqdm(range(config["num_clients"])):
    w_node = client_trainers[i].model.state_dict()
    norm_diff = np.infty
    new_cluster_id = 0
    for cluster_id in cluster_map.keys():
        w_cluster = cluster_trainers[cluster_id].model.state_dict()
        curr_norm_diff = model_weights_diff(w_node, w_cluster)
        if norm_diff > curr_norm_diff:
            new_cluster_id = cluster_id
            norm_diff = curr_norm_diff
    
    cluster_map_recluster[new_cluster_id].append(i)
cluster_map = cluster_map_recluster


G = nx.Graph()
G.add_nodes_from(cluster_map.keys())

all_pairs = list(itertools.combinations(cluster_map.keys(),2))
for pair in tqdm(all_pairs):
    w_1  = cluster_trainers[pair[0]].model.state_dict()
    w_2 = cluster_trainers[pair[1]].model.state_dict()
    norm_diff = model_weights_diff(w_1, w_2)
    if norm_diff < thresh:
        G.add_edge(pair[0], pair[1])
        
clusters = list(nx.algorithms.clique.enumerate_all_cliques(G))


100%|██████████| 12246/12246 [00:14<00:00, 820.48it/s]


In [85]:
cluster_map_new = {}
for i in range(len(clusters)):
    cluster_map_new[i] = []
    for j in clusters[i]:
        cluster_map_new[i] += cluster_map[j]
cluster_map = cluster_map_new

In [88]:
len(clusters)

668